# Evaluación de Métricas Temporales para SPMV

Este notebook evalúa el impacto del tiempo de ejecución en la calidad de los modelos de clasificación para SPMV.

In [ ]:
%%capture
!pip install odfpy
!pip install ultralytics
!git clone https://github.com/DiazCerecetto/SPMV.git

import sys
sys.path.insert(0, '/content/SPMV')

In [ ]:
import pandas as pd
import numpy as np
from spmv.config import Config
from spmv.dataset import DatasetManager
from spmv.images import ImageManager
from spmv.time_metrics import TimeMetricsEvaluator

# Initialize configuration and managers
config = Config()
image_manager = ImageManager()
dataset_manager = DatasetManager(config, image_manager)
time_evaluator = TimeMetricsEvaluator(config)

In [ ]:
# Load datasets
X_train, X_val, X_test, y_train, y_val, y_test = dataset_manager.leer_datasets()

# Load original data for time information
data_original = pd.read_csv(config.ARCHIVO_ENTRADA)

print("Shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"\nClase más frecuente en train: {y_train.mode()[0]}")

## 1. Evaluación usando predicción de clase mayoritaria

In [ ]:
majority_metrics = time_evaluator.evaluate_majority_class(y_test, data_original)

print("Métricas para predicción de clase mayoritaria:")
print(f"Accuracy: {majority_metrics['accuracy']:.4f}")
print(f"F1 (macro): {majority_metrics['f1_macro']:.4f}")
print(f"Penalización temporal promedio: {majority_metrics['avg_time_penalty']:.4f}")
print(f"Accuracy ponderado por tiempo: {majority_metrics['time_weighted_accuracy']:.4f}")

## 2. Evaluación usando Random Forest con todas las características

In [ ]:
rf_metrics = time_evaluator.evaluate_random_forest(
    X_train, y_train, X_test, y_test, data_original
)

print("\nMétricas para Random Forest:")
print(f"Accuracy: {rf_metrics['accuracy']:.4f}")
print(f"F1 (macro): {rf_metrics['f1_macro']:.4f}")
print(f"Penalización temporal promedio: {rf_metrics['avg_time_penalty']:.4f}")
print(f"Accuracy ponderado por tiempo: {rf_metrics['time_weighted_accuracy']:.4f}")

## 3. Evaluación usando Random Forest solo con características HOG

In [ ]:
# Identificar columnas HOG
hog_columns = [col for col in X_train.columns if 'hog' in col.lower()]
print(f"Número de características HOG: {len(hog_columns)}")

hog_metrics = time_evaluator.evaluate_hog_random_forest(
    X_train, y_train, X_test, y_test, data_original, hog_columns
)

print("\nMétricas para Random Forest con HOG:")
print(f"Accuracy: {hog_metrics['accuracy']:.4f}")
print(f"F1 (macro): {hog_metrics['f1_macro']:.4f}")
print(f"Penalización temporal promedio: {hog_metrics['avg_time_penalty']:.4f}")
print(f"Accuracy ponderado por tiempo: {hog_metrics['time_weighted_accuracy']:.4f}")

## Comparación de Resultados

In [ ]:
results = pd.DataFrame({
    'Método': ['Clase Mayoritaria', 'Random Forest', 'RF + HOG'],
    'Accuracy': [
        majority_metrics['accuracy'],
        rf_metrics['accuracy'],
        hog_metrics['accuracy']
    ],
    'F1 (macro)': [
        majority_metrics['f1_macro'],
        rf_metrics['f1_macro'],
        hog_metrics['f1_macro']
    ],
    'Penalización Temporal': [
        majority_metrics['avg_time_penalty'],
        rf_metrics['avg_time_penalty'],
        hog_metrics['avg_time_penalty']
    ],
    'Accuracy Temporal': [
        majority_metrics['time_weighted_accuracy'],
        rf_metrics['time_weighted_accuracy'],
        hog_metrics['time_weighted_accuracy']
    ]
})

results.style.format({
    'Accuracy': '{:.4f}',
    'F1 (macro)': '{:.4f}',
    'Penalización Temporal': '{:.4f}',
    'Accuracy Temporal': '{:.4f}'
})